# MNE Groups Data Discovery Challenge - MUR team

### Libraries

In [1]:
import os
import re
import random
import time
import PyPDF2
import requests
import pandas as pd
import yaml
import tempfile
from bs4 import BeautifulSoup
from googlesearch import search
from urllib.parse import urlparse

### Configuration

In [2]:
def load_config(config_file='config.yaml'):
    """
    Loads configuration parameters from a YAML file.

    Parameters:
        config_file (str): The path to the YAML configuration file.

    Returns:
        dict: A dictionary containing the loaded configuration parameters.
    """
    with open(config_file, 'r') as f:
        return yaml.safe_load(f)

In [3]:
config = load_config()

# --- Global Parameters from Config ---
DATA_PATH = config['data_path']
INPUT_FILENAME = config['input_filename']
OUTPUT_FILENAME = config['output_filename']
KEYWORDS_REPORT = config['keywords_report']
KEYWORDS_FINANCIAL = config['keywords_financial']
KEYWORDS_DISCOVERY = config['keywords_discovery']
VALID_YEARS = [str(year) for year in config['valid_years']]
PAUSE_TIME_MIN = config['pause_time_min']
PAUSE_TIME_MAX = config['pause_time_max']
SEARCH_TIMEOUT = config['search_timeout']
LONG_PAUSE_GROUPS_INTERVAL = config['long_pause_groups_interval']
LONG_PAUSE_TIME = config['long_pause_time']

### Functions

In [4]:
def is_pdf_url(url: str) -> bool:
    """
    Checks if a given URL points to a PDF file.

    Parameters:
        url (str): The URL to check.

    Returns:
        bool: True if the URL ends with '.pdf' (case-insensitive), False otherwise.
    """
    return url.lower().endswith(".pdf")

In [5]:
def extract_year(text: str, valid_years: list) -> str or None:
    """
    Searches for the first valid year from a list within a given text.

    Parameters:
        text (str): The text content to search within.
        valid_years (list): A list of year strings to look for.

    Returns:
        str or None: The first year found as a string, or None if no valid year is found.
    """
    if text is None:
        return None
    for year in valid_years:
        if str(year) in text:
            return year
    return None

In [6]:
def extract_year_from_pdf_url(url: str, valid_years: list, group_id: str or int, timeout: int = SEARCH_TIMEOUT) -> str or None:
    """
    Downloads a PDF from the given URL, extracts text from the first page,
    looks for a matching year, and then deletes the temporary file.

    Parameters:
        url (str): The URL of the PDF to download.
        valid_years (list): A list of year strings to search for.
        group_id (str or int): An identifier used to generate a unique temporary filename.
        timeout (int): Timeout in seconds for the HTTP request.

    Returns:
        str or None: The year found in the PDF as a string, or None if no year is found or an error occurs.
    """
    if not url:
        return None

    tmp_filename = os.path.join(tempfile.gettempdir(), f"temp_pdf_{group_id}.pdf")

    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                          "AppleWebKit/537.36 (KHTML, like Gecko) "
                          "Chrome/124.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        with open(tmp_filename, "wb") as f:
            f.write(response.content)

        with open(tmp_filename, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            if reader.is_encrypted:
                return None
            if len(reader.pages) > 0:
                text = reader.pages[0].extract_text() or ""
                text = text.lower()
                for year in valid_years:
                    if str(year) in text:
                        return year
                    
    except Exception as e:
        print(f"An unexpected error occurred while processing PDF from {url}: {e}")
    finally:
        if os.path.exists(tmp_filename):
            try:
                os.remove(tmp_filename)
            except Exception as cleanup_error:
                print(f"Warning: could not delete temporary file {tmp_filename}: {cleanup_error}")
    return None

In [7]:
def get_domain(url: str) -> str:
    """
    Extracts the domain name from a URL.

    Parameters:
        url (str): The input URL.

    Returns:
        str: The domain name (netloc), or an empty string if the URL is invalid.
    """
    try:
        return urlparse(url).netloc
    except:
        return ""

In [8]:
def check_isin(isin_code: str) -> bool:
    """
    Verifies if a given string is a valid ISIN code, including the check digit calculation.

    Parameters:
        isin_code (str): The ISIN code to validate.

    Returns:
        bool: True if the ISIN code is valid, False otherwise.
    """
    if not isinstance(isin_code, str) or len(isin_code) != 12:
        return False

    isin_code_upper = isin_code.upper()
    if not isin_code_upper[:2].isalpha() or not isin_code_upper[2:].isalnum():
        return False

    converted_digits = []
    for char in isin_code_upper:
        if '0' <= char <= '9':
            converted_digits.append(int(char))
        elif 'A' <= char <= 'Z':
            # Convert letters to numbers (A=10, B=11, ..., Z=35) and then split into digits
            converted_digits.extend(divmod(ord(char) - ord('A') + 10, 10))
        else:
            return False 

    total_sum = 0
    # Luhn algorithm: process from right to left
    for i in range(len(converted_digits) - 1, -1, -1):
        digit = converted_digits[i]
        if (len(converted_digits) - 1 - i) % 2 == 1:
            doubled_digit = digit * 2
            if doubled_digit > 9:
                total_sum += (doubled_digit % 10) + (doubled_digit // 10)
            else:
                total_sum += doubled_digit
        else:
            total_sum += digit
    return total_sum % 10 == 0

In [9]:
def search_isin(name: str, max_results: int = 10, timeout: int = SEARCH_TIMEOUT) -> str or None:
    """
    Searches for ISIN codes related to a given group name, validates them,
    and returns the first valid one found.

    Parameters:
        name (str): The name of the group to search for.
        max_results (int): The maximum number of search results to check.
        timeout (int): Timeout in seconds for the HTTP request.

    Returns:
        str or None: The first valid ISIN code found, or None if no valid ISIN is found.
    """
    query = f"{name} " + " OR ".join(["isin", "stock"])
    pause_time = random.uniform(PAUSE_TIME_MIN, PAUSE_TIME_MAX)
    time.sleep(pause_time)

    try:
        search_results = list(search(query, num_results=max_results))

        for url in search_results:
            try:
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                                  "Chrome/124.0.0.0 Safari/537.36"
                }
                response = requests.get(url, headers=headers, timeout=timeout)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")
                text = soup.get_text(separator=' ')

                processed_text = re.sub(r'[^a-z0-9\s]', ' ', text.lower())
                isin_pattern = r'\b([a-z]{2}[a-z0-9]{10})\b'
                match = re.search(isin_pattern, processed_text)

                if match and check_isin(match.group(1)):
                    found_isin = match.group(1).upper()
                    return found_isin

            except requests.exceptions.RequestException as e:
                print(f"Error getting URL {url} during ISIN search: {e}")
                continue
            except Exception as e:
                print(f"Error processing URL {url} during ISIN search: {e}")
                continue
        return None

    except Exception as e:
        print(f"Error during Google search for ISIN '{name}': {e}")
        print("Waiting before retrying Google search...")
        time.sleep(LONG_PAUSE_TIME)
        return None

In [10]:
def page_mentions_keywords(url: str, keywords: list, valid_years: list, timeout: int = SEARCH_TIMEOUT) -> tuple:
    """
    Fetches the page at the given URL and calculates a semantic score
    based on the presence of given keywords in the HTML content.
    Also attempts to extract a reference year from the text.

    Parameters:
        url (str): The URL of the web page.
        keywords (list): A list of keywords or lists of synonyms.
                         If an element is a list, it counts as 1 if at least one synonym is found.
        valid_years (list): A list of acceptable year strings to look for.
        timeout (int): Timeout in seconds for the HTTP request.

    Returns:
        tuple: A tuple containing:
               - score (int): The calculated semantic score.
               - refyear (str or None): The extracted reference year as a string, or None.
    """
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                 "AppleWebKit/537.36 (KHTML, like Gecko) "
                                 "Chrome/124.0.0.0 Safari/537.36"}
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator=' ').lower()
    except Exception as e:
        return 0, None

    score = 0
    for kw_group in keywords:
        if isinstance(kw_group, list):
            if any(k.lower() in text for k in kw_group):
                score += 1
        else:
            if kw_group.lower() in text:
                score += 1

    refyear = extract_year(text, valid_years)
    return score, refyear

In [11]:
def search_annual_report_url(name: str, max_results: int = 10) -> str or None:
    """
    Searches Google for an annual report URL for a group, prioritizing PDF files.

    Parameters:
        name (str): The group name.
        max_results (int): The maximum number of Google results to retrieve.

    Returns:
        str or None: The URL of the preferred annual report (PDF if available, otherwise the first result),
                     or None if no relevant URL is found.
    """
    query = f"{name} " + " ".join(KEYWORDS_REPORT)
    pause_time = random.uniform(PAUSE_TIME_MIN, PAUSE_TIME_MAX)
    time.sleep(pause_time)

    try:
        urls = list(search(query, lang="en", num_results=max_results))
        if not urls:
            return None
        preferred = next((u for u in urls if is_pdf_url(u)), urls[0])
        return preferred
    except Exception as e:
        print(f"Error during Google search for annual report '{name}': {e}")
        print("Waiting before retrying Google search...")
        time.sleep(LONG_PAUSE_TIME)
        return None

In [12]:
def search_other_info_urls(name: str, keywords: list, valid_years: list, max_results: int = 20, max_unique: int = 5, isin_code: str = None) -> list:
    """
    Searches Google for financial information URLs related to a group.

    For each valid URL, the function:
    - Ensures domain uniqueness to avoid multiple links from the same site.
    - Excludes PDF files from the search results.
    - Calculates a semantic score based on keyword presence.
    - Extracts a reference year from the HTML content.

    Parameters:
        name (str): The group name.
        keywords (list): A list of keywords or keyword groups for semantic scoring.
        valid_years (list): A list of valid reference years to search for.
        max_results (int): The total number of Google search results to retrieve.
        max_unique (int): The maximum number of unique domain URLs to return, sorted by score.
        isin_code (str, optional): An ISIN code to include in the search query for better relevance.

    Returns:
        list of tuples: A list where each tuple contains (URL (str), SCORE (int), REFYEAR (str or None)).
                        The list is sorted by SCORE in descending order.
    """
    query = f"{name} " + " OR ".join(KEYWORDS_FINANCIAL)
    if isin_code:
        query = query + " OR " + f"{isin_code}"

    seen_domains = set()
    urls_info = []
    pause_time = random.uniform(PAUSE_TIME_MIN, PAUSE_TIME_MAX)
    time.sleep(pause_time)

    try:
        results = list(search(query, lang="en", num_results=max_results))

        for url in results:
            domain = get_domain(url)
            if (domain in seen_domains) or is_pdf_url(url):
                continue
            seen_domains.add(domain)

            score, refyear = page_mentions_keywords(url, keywords, valid_years)
            if refyear:
                urls_info.append((url, score, refyear))

        # Sort by descending score and keep the top N unique results
        urls_info = sorted(urls_info, key=lambda x: x[1], reverse=True)[:max_unique]

        return urls_info
    except Exception as e:
        print(f"Error during Google search for other info '{name}': {e}")
        print("Waiting before retrying Google search...")
        time.sleep(LONG_PAUSE_TIME)
        return []

In [ ]:
def run_discovery():
    """
    Main function to run the discovery process.
    It reads group data, performs searches for ISIN codes, annual reports,
    and other financial information, and then saves the enriched data to a CSV file.
    """
    input_file_path = os.path.join("..", DATA_PATH, INPUT_FILENAME)
    output_file_path = os.path.join("..", DATA_PATH, OUTPUT_FILENAME)

    if not os.path.exists(os.path.join("..", DATA_PATH)):
        os.makedirs(os.path.join("..", DATA_PATH))
        print(f"Created data directory: {DATA_PATH}")

    try:
        df_original = pd.read_csv(input_file_path, sep=';')
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_file_path}. Please ensure it exists.")
        return
    except Exception as e:
        print(f"Error reading input CSV file: {e}")
        return

    df_original["SCORE"] = None  # Add SCORE column for semantic matching
    final_rows = []
    
    count = 0
    for group_id, group_df in df_original.groupby("ID"):
        count += 1
        if count % LONG_PAUSE_GROUPS_INTERVAL == 0 and count > 0:
            print(f"Processed {count} groups. Pausing for {LONG_PAUSE_TIME} seconds to avoid blocking...")
            time.sleep(LONG_PAUSE_TIME)

        name = group_df["NAME"].iloc[0]
        print(f"\nProcessing Group ID: {group_id}, Group Name: {name}")

        isin_code = None
        annual_url = None
        other_urls_info = []

        try:
            # Run searches
            isin_code = search_isin(name)
            annual_url = search_annual_report_url(name)
            other_urls_info = search_other_info_urls(name, KEYWORDS_DISCOVERY, VALID_YEARS, isin_code=isin_code)

            # Assign URLs depending on row TYPE
            for _, row in group_df.iterrows():
                new_row = row.copy()
                if row["TYPE"] == "FIN_REP":
                    new_row["SRC"] = annual_url if annual_url else ""
                    if annual_url:
                        # Attempt to extract year from URL first
                        year_from_url = extract_year(annual_url, VALID_YEARS)
                        if year_from_url:
                            new_row["REFYEAR"] = year_from_url
                        else: # If no year in URL, try extracting from PDF content
                            new_row["REFYEAR"] = extract_year_from_pdf_url(annual_url, VALID_YEARS, group_id=row["ID"])
                    else:
                        new_row["REFYEAR"] = ""
                    new_row["SCORE"] = ""
                else: # For other types of information
                    if other_urls_info:
                        other_url, score, year = other_urls_info.pop(0)
                        new_row["SRC"] = other_url
                        new_row["REFYEAR"] = year if year else ""
                        new_row["SCORE"] = score
                    else:
                        new_row["SRC"] = ""
                        new_row["REFYEAR"] = ""
                        new_row["SCORE"] = ""
                final_rows.append(new_row)

        except Exception as e:
            print(f"An error occurred while processing group {name} (ID: {group_id}): {e}")
            # In case of failure, append empty rows to preserve structure for this group
            for _, row in group_df.iterrows():
                new_row = row.copy()
                new_row["SRC"] = ""
                new_row["REFYEAR"] = ""
                new_row["SCORE"] = ""
                final_rows.append(new_row)

    # Final result DataFrame
    df_final_score = pd.DataFrame(final_rows)
    
    df_final = df_final_score.drop('SCORE', axis=1)

    # Save to output CSV
    try:
        df_final.to_csv(output_file_path, sep=";", index=False)
        print(f"\nProcess completed. Results saved to {output_file_path}")
    except Exception as e:
        print(f"Error saving output CSV file to {output_file_path}: {e}")

### Main

In [14]:
if __name__ == "__main__":
    run_discovery()


Processing Group ID: 18490453, Group Name: ADECCO GROUP AG

Processing Group ID: 18503381, Group Name: JOHNSON CONTROLS INTERNATIONAL PUBLIC LIMITED COMPANY

Processing Group ID: 18503440, Group Name: MAGNA INTERNATIONAL INC

Processing Group ID: 18511746, Group Name: PUBLICIS GROUPE S A

Processing Group ID: 18512048, Group Name: GEBR. KNAUF
Error getting URL https://www.bloomberg.com/profile/company/7685199Z:GR during ISIN search: 403 Client Error: Forbidden for url: https://www.bloomberg.com/profile/company/7685199Z:GR
Error getting URL https://www.sec.gov/Archives/edgar/data/757011/000119312518188777/d594332dex991.htm during ISIN search: 403 Client Error: Forbidden for url: https://www.sec.gov/Archives/edgar/data/757011/000119312518188777/d594332dex991.htm
Error getting URL https://www.investing.com/equities/usg-corp during ISIN search: 403 Client Error: Forbidden for url: https://www.investing.com/equities/usg-corp

Processing Group ID: 18519151, Group Name: BRIDGESTONE CORPORATI

incorrect startxref pointer(1)



Processing Group ID: 18736044, Group Name: SECURITAS AB

Processing Group ID: 18742207, Group Name: AIRBUS SE

Processing Group ID: 18745703, Group Name: ETEX
Error getting URL https://www.investing.com/equities/etex-sa during ISIN search: 403 Client Error: Forbidden for url: https://www.investing.com/equities/etex-sa
An unexpected error occurred while processing PDF from https://www.etexgroup.com/en/investors/annual-reports/: EOF marker not found

Processing Group ID: 18748693, Group Name: SIEMENS
Processed 50 groups. Pausing for 120 seconds to avoid blocking...

Processing Group ID: 18754669, Group Name: VODAFONE GROUP PLC

Processing Group ID: 18755204, Group Name: HBIS GROUP CO. LTD.
Error getting URL https://www.reuters.com/markets/companies/000709.SZ/ during ISIN search: 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/markets/companies/000709.SZ/
Error getting URL https://www.investing.com/equities/hebei-steel-a during ISIN search: 403 Client Error: Forbidden f

incorrect startxref pointer(1)



Processing Group ID: 18767600, Group Name: KRONES

Processing Group ID: 18773773, Group Name: KONECRANES

Processing Group ID: 18774479, Group Name: SANDVIK AKTIEBOLAG

Processing Group ID: 18774712, Group Name: SKANSKA AB
Error getting URL https://www.marketwatch.com/investing/stock/ska.b?countrycode=se during ISIN search: 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/investing/stock/ska.b?countrycode=se

Processing Group ID: 18774867, Group Name: SANOFI

Processing Group ID: 18778752, Group Name: WURTH

Processing Group ID: 18784770, Group Name: AKTIEBOLAGET VOLVO

Processing Group ID: 18796310, Group Name: TOTALENERGIES SE

Processing Group ID: 18802249, Group Name: BRITISH AMERICAN TOBACCO PLC

Processing Group ID: 18803013, Group Name: NIPPON STEEL CORPORATION
Error getting URL https://www.investing.com/equities/nippon-steel-sumitomo-metal-corp. during ISIN search: 403 Client Error: Forbidden for url: https://www.investing.com/equities/nippon-steel-sumitom

incorrect startxref pointer(1)



Processing Group ID: 18818842, Group Name: IWG PLC

Processing Group ID: 18831586, Group Name: SAP
Error getting URL https://www.sap.com/investors/en/stock/basic-data.html during ISIN search: 403 Client Error: Forbidden for url: https://www.sap.com/investors/en/stock/basic-data.html

Processing Group ID: 18839233, Group Name: ORKLA ASA

Processing Group ID: 18850720, Group Name: ENEL SPA

Processing Group ID: 18853782, Group Name: SHV HOLDINGS NV
Error getting URL https://www.bloomberg.com/profile/company/SHV:NA during ISIN search: 403 Client Error: Forbidden for url: https://www.bloomberg.com/profile/company/SHV:NA
Error getting URL https://www.marketwatch.com/investing/fund/shv/holdings during ISIN search: 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/investing/fund/shv/holdings

Processing Group ID: 18854880, Group Name: BMW
Processed 75 groups. Pausing for 120 seconds to avoid blocking...

Processing Group ID: 18858533, Group Name: THALES
An unexpected erro

incorrect startxref pointer(1)


An unexpected error occurred while processing PDF from https://regnskaber.cvrapi.dk/xhtml/59380412/amNsb3VkczovLzAzL2I5LzI3LzVkL2M2L2RlNTgtNDNjNS1hMjY3LWZhZDIyMGY2YjlkYw.pdf: xref table read error

Processing Group ID: 19266518, Group Name: WIENERBERGER GROUP
Error getting URL https://www.investing.com/equities/wienerberger during ISIN search: 403 Client Error: Forbidden for url: https://www.investing.com/equities/wienerberger

Processing Group ID: 19275814, Group Name: VESTAS WIND SYSTEMS A/S
Error getting URL https://www.investing.com/equities/vestas-wind during ISIN search: 403 Client Error: Forbidden for url: https://www.investing.com/equities/vestas-wind

Processing Group ID: 19276100, Group Name: YARA INTERNATIONAL ASA

Processing Group ID: 19286023, Group Name: MITSUBISHI CORPORATION

Processing Group ID: 19287986, Group Name: ZF FRIEDRICHSHAFEN

Processing Group ID: 19297963, Group Name: KRKA TOVARNA ZDRAVIL DD NOVO MESTO
Error getting URL https://www.gpw.pl/company-factsheet?i